# Technology

gdsfactory has a default `config.yml` file that inclues all the defaults for the generic Technology.

[YAML](https://www.cloudbees.com/blog/yaml-tutorial-everything-you-need-get-started/) is a human friendly format

To overwrite any of the defaults you can just create a `config.yml` in the current directory of your project.

The `conig.yml` can define:

- Layers



## Layers

A GDS has different layers to describe the different fabrication process steps.

GDS layers have 2 integer numbers: GDSlayer, GDSpurpose

Klayout shows Layers with a color, style and transparency when showing GDS layouts.

In [ ]:
import omegaconf
import dataclasses
import pp
from pp.layers import preview_layerset
from pp.config import print_config

In [ ]:
print_config()

In [ ]:
layer_wg = pp.LAYER.WG
print(layer_wg)

In [ ]:
c = preview_layerset()
c

## TECH

As you can see in `pdk.tech` you can define all your technology constants in YAML. 
This is a convenient way to group all the specific constants for a technology. 
For example, Lets say we want to add a new layer and a new waveguide definition in YAML format in `config.yml`.

In [ ]:
import pp

pp.tech('waveguide.pn')

In [ ]:
c = pp.components.straight(**pp.TECH.waveguide.pn)
c

## PDK

How can you customize components for a particular technology?

You can customize the `components` field.

In [ ]:
import pp

cross_section_settings=pp.TECH.waveguide.nitride
c = pp.components.straight(width=2, cross_section_settings=cross_section_settings)
c

In [ ]:
cross_section_settings=pp.TECH.waveguide.nitride
c = pp.components.straight(width=2, cross_section_settings=cross_section_settings)
gc = pp.components.grating_coupler_elliptical_te(layer=pp.TECH.waveguide.nitride.layer, wg_width=pp.TECH.waveguide.nitride.width)
cc = pp.routing.add_fiber_single(component=c, grating_coupler=gc, cross_section_settings=cross_section_settings)
cc